**This notebook to answers StackOverflow and Issue Questions**

# Imports

In [ ]:
# Check version runing on Jupyter notebook
from platform import python_version
import sys

print('Python Version : ', python_version())
print('Python Path : ', sys.executable)

In [ ]:
import sys
sys.path.append('../')

In [ ]:
%load_ext autoreload
%autoreload 2
from swtloc import SWTLocalizer
from swtloc.configs import (IMAGE_ORIGINAL, 
                            IMAGE_GRAYSCALE,
                            IMAGE_EDGED,
                            IMAGE_SWT_TRANSFORMED,
                            IMAGE_CONNECTED_COMPONENTS_3C,
                            IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS,
                            IMAGE_CONNECTED_COMPONENTS_PRUNED_3C,
                            IMAGE_PRUNED_3C_LETTER_LOCALIZATIONS,
                            IMAGE_ORIGINAL_LETTER_LOCALIZATIONS,
                            IMAGE_ORIGINAL_MASKED_LETTER_LOCALIZATIONS,
                            IMAGE_PRUNED_3C_WORD_LOCALIZATIONS,
                            IMAGE_ORIGINAL_WORD_LOCALIZATIONS,
                            IMAGE_ORIGINAL_MASKED_WORD_LOCALIZATIONS)

In [ ]:
from cv2 import cv2
import numpy as np
cv2.__version__

# Raw Datapaths and Variable Initialisation

In [ ]:
rawimage_path = 'images/'

# Q1
https://stackoverflow.com/questions/40443988/python-opencv-ocr-image-segmentation

In [ ]:
imgpath = [rawimage_path+'so1/so1_img1.jpg', rawimage_path+'so1/so1_img2.jpg']
respath = rawimage_path+'so1/results/'

swtl = SWTLocalizer(image_paths=imgpath)
swtImgObj = swtl.swtimages[1]
print(swtImgObj)

In [ ]:
# Perform SWT Transformation with numba engine
swt_mat = swtImgObj.transformImage(text_mode='lb_df', gaussian_blurr=False, 
                                   minimum_stroke_width=3, maximum_stroke_width=12,
                                   maximum_angle_deviation=np.pi/2)

In [ ]:
# Localized Letters
localized_letters = swtImgObj.localizeLetters(minimum_pixels_per_cc=10,
                                              localize_by='min_bbox')
localized_words =  swtImgObj.localizeWords(localize_by='bbox')

**Generating Images**

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL, IMAGE_GRAYSCALE, IMAGE_EDGED, IMAGE_SWT_TRANSFORMED],
                    plot_title='SWT', plot_sup_title=f'\nTransform Time - {swtImgObj.transform_time}',
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_CONNECTED_COMPONENTS_3C,
                                IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS,
                                IMAGE_PRUNED_3C_LETTER_LOCALIZATIONS,
                                IMAGE_ORIGINAL_MASKED_LETTER_LOCALIZATIONS],
                     plot_title='Letter Localizations\n',
                     plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizeletters.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_PRUNED_3C_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_MASKED_WORD_LOCALIZATIONS],
                    plot_title='Word Localizations\n',
                    plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizewords.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

# Q2

https://stackoverflow.com/questions/4837124/stroke-width-transform-swt-implementation-java-c

In [ ]:
from swtloc import SWTLocalizer

imgpath = rawimage_path+'so2/so2_img1.jpg'
respath = rawimage_path+'so2/results/'
swtl = SWTLocalizer(image_paths=imgpath)
swtImgObj = swtl.swtimages[0]
print(swtImgObj)


In [ ]:
# Perform SWT Transform
swt_mat = swtImgObj.transformImage(auto_canny_sigma=1.0, minimum_stroke_width=3,
                                   maximum_stroke_width=20, maximum_angle_deviation=np.pi/6)

In [ ]:
# Find and Prune Connected Components
localized_letters = swtImgObj.localizeLetters()

In [ ]:
# Find and Prune Connected Components
localized_words = swtImgObj.localizeWords(acceptable_stroke_width_ratio=2.5)

**Generating Images**

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL, IMAGE_GRAYSCALE, IMAGE_EDGED, IMAGE_SWT_TRANSFORMED],
                    plot_title='SWT', plot_sup_title=f'\nTransform Time - {swtImgObj.transform_time}',
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_CONNECTED_COMPONENTS_3C,
                                IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS,
                                IMAGE_PRUNED_3C_LETTER_LOCALIZATIONS,
                                IMAGE_ORIGINAL_MASKED_LETTER_LOCALIZATIONS],
                     plot_title='Letter Localizations\n',
                     plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizeletters.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_PRUNED_3C_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_MASKED_WORD_LOCALIZATIONS],
                    plot_title='Word Localizations\n',
                    plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizewords.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

# Q3
https://stackoverflow.com/questions/31667709/can-swt-stroke-width-transform-help-ocr-with-screenshots

In [ ]:
from cv2 import cv2

def resize_maintinaAR(image, width=1.0, height=1.0, inter=cv2.INTER_AREA, mode='proportion'):
    """
    A function to resize the image based on the params.
    # Adopted from : https://stackoverflow.com/a/55306956/6297658

    Arguments
    ------------------------------
        image : Original Image, np.ndarray
            Image to resize
        
        width(Optional) : int or float.
            How much to resize based on the width.
        
        height(Optional) : int or float
            How much to resize based on the height.
        
        inter(Optional) : opencv interpolation mode
        
        mode(Optional) : One of 'proportion' or 'actual'
            Which mode to resize the image in.
    Returns
    ------------------------------
    Resized image
    """
    dim = None
    (h, w) = image.shape[:2]

    if mode == 'proportion':
        width *= w
        width = int(width)
        height *= h
        height = int(height)

    # Return original image if no need to resize
    if width is None and height is None:
        return image

    # We are resizing height if width is none
    if width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # We are resizing width if height is none
    else:
        # Calculate the ratio of the width and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Return the resized image
    return cv2.resize(image, dim, interpolation=inter)


In [ ]:
from swtloc import SWTLocalizer

imgpath = rawimage_path+'so3/so3_img1.png'
r_imgpath = rawimage_path+'so3/so3_img11.jpg'
respath = rawimage_path+'so3/results/'

orig_img = cv2.imread(imgpath)
resized_img = resize_maintinaAR(orig_img, width=2.0)
print(f'Shape changed from {orig_img.shape} -> {resized_img.shape}')

In [ ]:
swtl = SWTLocalizer(images=resized_img)
swtImgObj = swtl.swtimages[0]
print(swtImgObj)

In [ ]:
swt_mat = swtImgObj.transformImage(minimum_stroke_width=3, maximum_stroke_width=10, 
                                   maximum_angle_deviation=np.pi/6)

In [ ]:
# Localizations # TODO : Memory Error
# localized_letters = swtImgObj.localizeLetters()

**Generating Images**

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL, IMAGE_GRAYSCALE, IMAGE_EDGED, IMAGE_SWT_TRANSFORMED],
                    plot_title='SWT', plot_sup_title=f'\nTransform Time - {swtImgObj.transform_time}',
                    save_fig=True, save_dir=respath, dpi=300)

# Q4
https://stackoverflow.com/questions/23506105/extracting-text-opencv

In [ ]:
import os
imgpaths = [rawimage_path+'so4/so4_img1.png',
            rawimage_path+'so4/so4_img2.jpg',
            rawimage_path+'so4/so4_img3.jpg']
respath = rawimage_path+'so4/results/'

In [ ]:
from swtloc import SWTLocalizer
from swtloc.configs import IMAGE_ORIGINAL

swtl = SWTLocalizer(image_paths=imgpaths)
print(swtl.swtimages)

**First Image**

In [ ]:
swtImgObj = swtl.swtimages[0]
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL])

In [ ]:
swt_mat = swtImgObj.transformImage(text_mode='lb_df', minimum_stroke_width=3, maximum_stroke_width=25,
                                   maximum_angle_deviation=np.pi/4, gaussian_blurr_kernel=(3, 3))

In [ ]:
# Localizations
localized_letters = swtImgObj.localizeLetters(minimum_pixels_per_cc=80,
                                              maximum_pixels_per_cc=2000)
localized_words = swtImgObj.localizeWords()

**Second Image**

In [ ]:
swtImgObj = swtl.swtimages[1]
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL])

In [ ]:
swt_mat = swtImgObj.transformImage(text_mode='db_lf', minimum_stroke_width=3, maximum_stroke_width=25,
                                   maximum_angle_deviation=np.pi/5, gaussian_blurr_kernel=(9, 9))

In [ ]:
# Localizations
localized_letters = swtImgObj.localizeLetters(minimum_pixels_per_cc=70,
                                              maximum_pixels_per_cc=3500,
                                              acceptable_aspect_ratio=0.05)
localized_words = swtImgObj.localizeWords(localize_by='bbox')

**Third Image**

In [ ]:
swtImgObj = swtl.swtimages[2]
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL])

In [ ]:
swt_mat = swtImgObj.transformImage(text_mode='lb_df', minimum_stroke_width=3, maximum_stroke_width=25,
                                   maximum_angle_deviation=np.pi/3, gaussian_blurr_kernel=(5, 5))

In [ ]:
# Localizations
localized_letters = swtImgObj.localizeLetters(minimum_pixels_per_cc=50,
                                              maximum_pixels_per_cc=2500)
localized_words = swtImgObj.localizeWords(localize_by='bbox')

**Saving for each image**

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL, IMAGE_GRAYSCALE, IMAGE_EDGED, IMAGE_SWT_TRANSFORMED],
                    plot_title='SWT', plot_sup_title=f'\nTransform Time - {swtImgObj.transform_time}',
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_CONNECTED_COMPONENTS_3C,
                                IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS,
                                IMAGE_PRUNED_3C_LETTER_LOCALIZATIONS,
                                IMAGE_ORIGINAL_MASKED_LETTER_LOCALIZATIONS],
                     plot_title='Letter Localizations\n',
                     plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizeletters.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_PRUNED_3C_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_MASKED_WORD_LOCALIZATIONS],
                    plot_title='Word Localizations\n',
                    plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizewords.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

# Q5
https://stackoverflow.com/questions/46533389/find-the-width-of-an-ink-stroke-in-an-image-using-opencv-c/46533420

In [ ]:
from swtloc import SWTLocalizer

imgpath = rawimage_path+'so5/so5_img1.jpg'
respath = rawimage_path+'so5/results/'
swtl = SWTLocalizer(image_paths=imgpath)
swtImgObj = swtl.swtimages[0]
swt_mat = swtImgObj.transformImage(auto_canny_sigma=1.0, gaussian_blurr=False,
                                   minimum_stroke_width=3, maximum_stroke_width=50,
                                   maximum_angle_deviation=np.pi/3)

In [ ]:
# Localize Letters
localized_letters = swtImgObj.localizeLetters()
localized_words = swtImgObj.localizeWords()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

all_sws = []
for letter_label, letter in localized_letters.items():
    all_sws.append(letter.stroke_widths_mean)
sns.displot(all_sws, bins=31)
plt.savefig(respath+'sw_distribution.jpg')

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL, IMAGE_GRAYSCALE, IMAGE_EDGED, IMAGE_SWT_TRANSFORMED],
                    plot_title='SWT', plot_sup_title=f'\nTransform Time - {swtImgObj.transform_time}',
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_CONNECTED_COMPONENTS_3C,
                                IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS,
                                IMAGE_PRUNED_3C_LETTER_LOCALIZATIONS,
                                IMAGE_ORIGINAL_MASKED_LETTER_LOCALIZATIONS],
                     plot_title='Letter Localizations\n',
                     plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizeletters.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_PRUNED_3C_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_MASKED_WORD_LOCALIZATIONS],
                    plot_title='Word Localizations\n',
                    plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizewords.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

# Q6
Bubble BBOX issue : Issue #10

In [ ]:
from swtloc import SWTLocalizer

imgpath = rawimage_path+'issue10/issue10.jpg'
respath = rawimage_path+'issue10/results/'
swtl = SWTLocalizer(image_paths=imgpath)
swtImgObj = swtl.swtimages[0]
# Stroke Width Transform
swt_mat = swtImgObj.transformImage(text_mode='db_lf', auto_canny_sigma=1.0,
                                   minimum_stroke_width=3, maximum_stroke_width=200,
                                   maximum_angle_deviation=np.pi/8,
                                   gaussian_blurr_kernel=(5, 5))

In [ ]:
# Localize Letters
localized_letters = swtImgObj.localizeLetters(maximum_pixels_per_cc=5_000)
# Localize Words
localized_words = swtImgObj.localizeWords(localize_by='bbox')

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL, IMAGE_GRAYSCALE, IMAGE_EDGED, IMAGE_SWT_TRANSFORMED],
                    plot_title='SWT', plot_sup_title=f'\nTransform Time - {swtImgObj.transform_time}',
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_CONNECTED_COMPONENTS_3C,
                                IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS,
                                IMAGE_PRUNED_3C_LETTER_LOCALIZATIONS,
                                IMAGE_ORIGINAL_MASKED_LETTER_LOCALIZATIONS],
                     plot_title='Letter Localizations\n',
                     plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizeletters.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_PRUNED_3C_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_MASKED_WORD_LOCALIZATIONS],
                    plot_title='Word Localizations\n',
                    plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizewords.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

# Q7

Issue BBOX Crop Feature Request : Issue 14

In [ ]:
from swtloc import SWTLocalizer

imgpath = rawimage_path+'issue14/issue14.png'
respath = rawimage_path+'issue14/results/'
swtl = SWTLocalizer(image_paths=imgpath)
swtImgObj = swtl.swtimages[0]
# Stroke Width Transform
swt_mat = swtImgObj.transformImage(text_mode='lb_df',
                                   minimum_stroke_width=2, maximum_stroke_width=20,
                                   maximum_angle_deviation=np.pi/4,
                                   gaussian_blurr_kernel=(7, 7),
                                   include_edges_in_swt=True)

In [ ]:
# Localize Letters
localized_letters = swtImgObj.localizeLetters(maximum_pixels_per_cc=5_000,
                                                acceptable_aspect_ratio=0.22,
                                                localize_by='min_bbox')
# Localize Words
localized_words = swtImgObj.localizeWords(localize_by='bubble')

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_ORIGINAL, IMAGE_GRAYSCALE, IMAGE_EDGED, IMAGE_SWT_TRANSFORMED],
                    plot_title='SWT', plot_sup_title=f'\nTransform Time - {swtImgObj.transform_time}',
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_CONNECTED_COMPONENTS_3C,
                                IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS,
                                IMAGE_PRUNED_3C_LETTER_LOCALIZATIONS,
                                IMAGE_ORIGINAL_MASKED_LETTER_LOCALIZATIONS],
                     plot_title='Letter Localizations\n',
                     plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizeletters.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

swtImgObj.showImage(image_codes=[IMAGE_PRUNED_3C_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_WORD_LOCALIZATIONS,
                                 IMAGE_ORIGINAL_MASKED_WORD_LOCALIZATIONS],
                    plot_title='Word Localizations\n',
                    plot_sup_title=rf"Localization Method : ${swtImgObj.cfg['swtimage.localizewords.localize_by']}$",
                    save_fig=True, save_dir=respath, dpi=300)

**Generating BBOX Crops**

In [ ]:
localized_words = swtImgObj.localizeWords(localize_by='bubble', display=False)
word_iterator = swtImgObj.wordIterator()

In [ ]:
word, _, _ = next(word_iterator)

In [ ]:
word_label = word.label
swtImgObj.saveCrop(save_path=respath, crop_of='words', crop_key=word_label,
                   crop_on=IMAGE_SWT_TRANSFORMED, crop_type='bubble')
swtImgObj.saveCrop(save_path=respath, crop_of='words', crop_key=word_label,
                   crop_on=IMAGE_ORIGINAL, crop_type='bbox')